# Lab 3 - OpenAI Agents SDK with streaming

In [18]:
from dotenv import load_dotenv
from agents import Agent, Runner
from openai.types.responses import ResponseTextDeltaEvent
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr

load_dotenv(override=True)

True

In [ ]:
name = "Kinjal Shah"
reader = PdfReader("info/KinjalShah_Resume.pdf")
resume = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        resume += text
profile = PdfReader("info/LinkedIn-Profile.pdf")
profile = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        profile += text

with open("info/ks_summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

instructions = f"""You represent the AI Digital Twin of a human called {name}. You are answering questions on {name}'s website, 
particularly questions related to {name}'s career, background, skills and experience. 
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. 
You are given a summary of {name}'s background,  LinkedIn profile and resumewhich you can use to answer questions. 
Be professional and engaging, as if talking to a potential client or future employer who came across the website. 
    You are friendly and amiable, and you introduce yourself as {name}'s Digital Twin.
{name} is Software Engineer Leader and a consultant.
He loves coding and experimenting with LLMs as well as agentic AI. He is also a very good project manager and product manager.
You chat with visitors on Kinjal's personal website. You answer questions about Kinjal's work.
If you don't know the answer, say so.
"""

instructions += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{profile}\n\n## Resume:\n{resume}\n\n"
instructions += f"With this context, please chat with the user, always staying in character as {name}."


In [20]:
agent = Agent(name="Twin", instructions=instructions, model="gpt-4.1-mini")

In [21]:
async def chat(message, history):
    messages = [{"role": prior["role"], "content": prior["content"]} for prior in history]  
    messages += [{"role": "user", "content": message}]
    response = await Runner.run(agent, messages)
    return response.final_output

In [22]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7865
* To create a public link, set `share=True` in `launch()`.


In [15]:
async def chat(message, history):
    messages = [{"role": prior["role"], "content": prior["content"]} for prior in history]  
    messages += [{"role": "user", "content": message}]
    response = Runner.run_streamed(agent, messages)
    reply = ""
    async for event in response.stream_events():
        if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
            reply += event.data.delta
            yield reply

In [16]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.
